In [1]:
import os

In [2]:
pwd

'c:\\Ineuron\\wine_quality_prediction\\research'

In [3]:
os.chdir("../")

In [4]:
pwd

'c:\\Ineuron\\wine_quality_prediction'

In [5]:
#prepare the entity

In [28]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_cloumn: str

In [7]:
#update the configuration manager in src config

In [29]:
from src.mlproject.constants import *
from src.mlproject.utils.common import read_yaml,create_directories

In [30]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH,
        schema_filepath= SCHEMA_FILE_PATH
        ):
            
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_cloumn = schema.name
        )
        return model_trainer_config 

In [15]:
#update the components

In [31]:
import os
from src.mlproject import logger
import pandas as pd
from sklearn.linear_model import ElasticNet
import joblib


In [32]:
class ModelTrainer:
    def __init__(self, config:ModelTrainerConfig):
        self.config = config
        

    def tarin(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)    


        train_X = train_data.drop([self.config.target_cloumn],axis=1)
        test_X = test_data.drop([self.config.target_cloumn],axis=1)
        train_y = train_data[[self.config.target_cloumn]]
        test_y = test_data[[self.config.target_cloumn]]

        lr=ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_X,train_y)

        joblib.dump(lr,os.path.join(self.config.root_dir, self.config.model_name))



In [20]:
#update the pipeline

In [33]:
try:
    config= ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.tarin()
except Exception as e:
    raise e    

[2024-01-20 19:34:35,812: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-20 19:34:35,814: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-20 19:34:35,820: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-20 19:34:35,822: INFO: common: created directory at: artifacts]
[2024-01-20 19:34:35,822: INFO: common: created directory at: artifacts/model_trainer]
